In [29]:
%load_ext sql

In [30]:
%config SqlMagic.displaylimit = 15
# %config SqlMagic.displaylimit = None

In [ ]:
import datetime as dt
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text

In [594]:
sys_log_db = "system_data_log.db"

sql_engine = create_engine("sqlite:///%s" % sys_log_db, echo=False)

In [189]:
def create_voltage_table(force=False):
    with sql_engine.connect() as sql_conn:
        if force:
            sql_text = text("""DROP TABLE IF EXISTS voltages;
                            """)
            sql_conn.execute(sql_text)
        # define schema
        sql_text = text("""
            CREATE TABLE IF NOT EXISTS voltages (
                Timestamp TEXT,
                Vmain FLOAT,
                Vmain_raw FLOAT,
                Vaux FLOAT,
                Vaux_raw FLOAT,
                PRIMARY KEY (Timestamp)
                )
        """)
        sql_conn.execute(sql_text)
        sql_conn.commit()

In [ ]:
create_voltage_table()

In [193]:
def add_voltage_log(Vmain, Vmain_raw, Vaux, Vaux_raw):
    with sql_engine.connect() as sql_conn:
        sql_text = text("""INSERT INTO voltages
                           VALUES (DateTime("now", "localtime"),
                                   :v_main,
                                   :v_main_raw,
                                   :v_aux,
                                   :v_aux_raw
                           );
                        """)
        sql_conn.execute(sql_text, {"v_main": Vmain,
                                    "v_main_raw": Vmain_raw,
                                    "v_aux": Vaux,
                                    "v_aux_raw": Vaux_raw})
        sql_conn.commit()

In [194]:
add_voltage_log(12.4, 12.8, 12.5, 12)

In [282]:
def get_voltages(voltage_type="Vmain", trailing_seconds=None):
    with sql_engine.connect() as sql_conn:
        sql_text = text(f"""SELECT Timestamp, {voltage_type}
                            FROM voltages
                         """)
        result_df = pd.read_sql(sql_text, con=sql_conn, index_col = "Timestamp", parse_dates=["Timestamp"])
    if trailing_seconds is not None:
        return result_df[result_df.index > dt.datetime.now() - dt.timedelta(seconds=trailing_seconds)]
    else:
        return result_df

In [595]:
def get_voltages(voltage_type=None, trailing_seconds=None):
    if voltage_type is None:
        cols = "*"
    else:
        cols = "Timestamp, %s" % voltage_type

    if trailing_seconds is not None:
        timestamp_now = dt.datetime.now()
        timestamp_trail = timestamp_now - dt.timedelta(seconds=trailing_seconds)
        timestamp_trail_str = timestamp_trail.strftime("%Y-%m-%d %H:%M:%S")
        time_filter = "WHERE Timestamp > %s" % timestamp_trail_str
    else:
        time_filter = ""

    with sql_engine.connect() as sql_conn:
        # sql_text = text(f"""SELECT Timestamp, {voltage_type}
        sql_text = text(f"""SELECT {cols}
                            FROM voltages
                            {time_filter}
                         """)
        result_df = pd.read_sql(sql_text, con=sql_conn, index_col = "Timestamp", parse_dates=["Timestamp"])
    return result_df

In [344]:
df = get_voltages("Vmain"); df

,Vmain
Timestamp,
2025-10-12 15:08:14,12.4
2025-10-12 15:08:21,12.4
2025-10-12 15:08:22,12.5


---

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
pandas.DataFrame.to_sql(sys_log_db, )

In [487]:
table_df = dl.get_charging(dt.datetime.now()); table_df

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,
2025-10-13 21:16:05,1,1,4.3,0.1,0.03
2025-10-13 21:16:45,1,1,4.3,0.1,0.03
2025-10-13 21:17:03,1,1,4.3,0.1,0.03
2025-10-13 21:17:50,1,1,4.3,0.1,0.03
2025-10-13 21:27:45,1,1,4.3,0.1,0.03
2025-10-13 21:28:15,1,1,4.3,0.1,0.03
2025-10-13 21:30:07,1,1,4.3,0.1,0.03


In [493]:
time_now = dt.datetime.now()

In [497]:
(time_now - table_df.index[-1]) > dt.timedelta(seconds=5)

True

In [498]:
table_df = dl.get_charging(dt.datetime.now(), 5); table_df

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,


In [499]:
len(table_df.index)

0

---

In [640]:
from importlib import reload
import class_def
reload(class_def)
dl = class_def.DataLogger()

In [593]:
dl._execute_sql("SELECT * FROM voltages", query=True)

,Vmain_raw,Vaux_raw
Timestamp,,
2025-10-17 20:41:13,12.298,12.909
2025-10-17 20:42:52,12.298,12.909
NaT,12.236,12.909
NaT,12.236,12.909
2025-10-17 21:09:41,12.220,12.909
...,...,...
2025-10-18 08:11:21,12.878,12.659
2025-10-18 08:11:22,12.894,12.659
2025-10-18 08:11:23,12.894,12.659


In [636]:
dl.get_voltages(dt.datetime.now(), 3000)

,Vmain_raw,Vaux_raw
Timestamp,,
2025-10-18 07:41:49,12.894,12.674
2025-10-18 07:41:50,12.894,12.674
2025-10-18 07:41:51,12.894,12.674
2025-10-18 07:41:52,12.894,12.659
2025-10-18 07:41:53,12.894,12.674
...,...,...
2025-10-18 08:31:44,12.894,12.643
2025-10-18 08:31:45,12.894,12.659
2025-10-18 08:31:46,12.894,12.643


In [637]:
dl.get_voltages(dt.datetime.now(), 1000, ["Vmain_raw"])

,Vmain_raw
Timestamp,
2025-10-18 08:15:10,12.894
2025-10-18 08:15:11,12.894
2025-10-18 08:15:12,12.894
2025-10-18 08:15:13,12.894
2025-10-18 08:15:14,12.894
...,...
2025-10-18 08:31:44,12.894
2025-10-18 08:31:45,12.894
2025-10-18 08:31:46,12.894


In [ ]:
dl.get_charging(dt.datetime.now())

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,
2025-10-17 20:42:52,0,1,8.266667,12.909,12.862
NaT,0,1,8.266667,12.909,12.878
NaT,0,1,8.266667,12.909,12.878
2025-10-17 21:09:41,0,1,8.533333,12.894,12.862
2025-10-17 21:09:42,0,1,8.266667,12.909,12.862
...,...,...,...,...,...
2025-10-18 07:49:00,1,1,12.533333,12.674,12.627
2025-10-18 07:49:01,1,1,12.533333,12.674,12.627
2025-10-18 07:49:02,1,1,8.533333,12.674,12.627


In [641]:
dl.get_signals(dt.datetime.now(), 1000)

,enable_sw,key_ACC,ecu_W,engine_on,network_conn,HAT_analog_0,HAT_analog_1,HAT_analog_2,HAT_input_0,HAT_input_1,HAT_input_2,HAT_relay_0,HAT_relay_1,HAT_relay_2
Timestamp,,,,,,,,,,,,,,
2025-10-18 08:16:43,1,1,0,0,home network,12.612,12.659,12.894,0,1,1,1,0,1
2025-10-18 08:16:44,1,1,0,0,home network,12.612,12.659,12.894,0,1,1,1,0,1
2025-10-18 08:16:45,1,1,0,0,home network,12.612,12.659,12.894,0,1,1,1,0,1
2025-10-18 08:16:46,1,1,0,0,home network,12.612,12.659,12.894,0,1,1,1,0,1
2025-10-18 08:16:47,1,1,0,0,home network,12.612,12.659,12.894,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-18 08:31:44,1,1,0,0,home network,12.612,12.643,12.894,0,1,1,1,0,1
2025-10-18 08:31:45,1,1,0,0,home network,12.612,12.659,12.894,0,1,1,1,0,1
2025-10-18 08:31:46,1,1,0,0,home network,12.612,12.643,12.894,0,1,1,1,0,1
